In [ ]:
 ! pip install pystac-client stackstac xarray dask[complete] rioxarray rasterio geopandas shapely numpy pandas scikit-learn pyproj

In [9]:
import os
import sys
from pathlib import Path
import xarray as xr


PROJECT_ROOT = Path.cwd()  # onde está o .ipynb
sys.path.append(str(PROJECT_ROOT))

from utils.database import fetch_vector_from_postgres


os.environ['DB_HOST'] = "localhost"
os.environ["DB_PORT"] = "5433"
os.environ['DB_NAME'] = "hackathon"
os.environ['DB_USER'] = "hack_user"
os.environ['DB_PASSWORD'] = "hack_pass"

PG_URI = "postgresql://{0}:{1}@{2}:{3}/{4}".format( os.environ['DB_USER'],
                                                    os.environ['DB_PASSWORD'],
                                                    os.environ['DB_HOST'],
                                                    os.environ['DB_PORT'],
                                                    os.environ['DB_NAME'])

STAC_URL = "https://landsatlook.usgs.gov/stac-server"
query ="SELECT id, luh_nm, ST_envelope(geom) geom FROM unidades_hidrograficas where luh_nm = 'Rio Jardim';"
UH = fetch_vector_from_postgres(PG_URI, query)
AOI_POLY = UH.iloc[0].geom.__geo_interface__
AOI_BBOX = UH.total_bounds.tolist()

DATA_INICIO_VAZIO = "2025-07-01"
DATA_FIM_VAZIO    = "2025-09-30"

In [11]:
# --- antes (que deu erro) ---
# s2A_idx = s2_indices(s2A).isel(time=0)
# s2B_idx = s2_indices(s2B).isel(time=0)
# s1A_ds  = s1_feats(s1A).isel(time=0)
# s1B_ds  = s1_feats(s1B).isel(time=0)

# --- depois (sem 'time') ---
def drop_time_coords(ds):
    # remove a dimensão/coord 'time' por completo
    if "time" in ds.dims:
        ds = ds.isel(time=0, drop=True)
    if "time" in ds.coords:
        ds = ds.reset_coords("time", drop=True)
    return ds

s2A_idx = drop_time_coords(s2_indices(s2A))
s2B_idx = drop_time_coords(s2_indices(s2B))
s1A_ds  = drop_time_coords(s1_feats(s1A))
s1B_ds  = drop_time_coords(s1_feats(s1B))

# alinhar grades (usa NDVI de S2 como referência)
ref = s2A_idx["NDVI"]
s2A_idx = s2A_idx.interp_like(ref)
s2B_idx = s2B_idx.interp_like(ref)
s1A_ds  = s1A_ds.interp_like(ref)
s1B_ds  = s1B_ds.interp_like(ref)

# merge seguro (sem coord 'time' conflitando)
import xarray as xr
A = xr.merge([s2A_idx, s1A_ds], compat="override", join="exact")
B = xr.merge([s2B_idx, s1B_ds], compat="override", join="exact")

# dNDVI instantâneo
dNDVI = (A["NDVI"] - B["NDVI"]).rename("dNDVI")



In [13]:
# -*- coding: utf-8 -*-
# Passo mínimo: 1 cena S2 mais recente -> NDVI -> GeoTIFF
from datetime import datetime
import numpy as np
import xarray as xr
from pystac_client import Client
import stackstac, rasterio
from rasterio.transform import Affine

# ===== Parâmetros =====
STAC_URL = "https://earth-search.aws.element84.com/v1"


# Janela do vazio (ajuste conforme calendário oficial)
DATA_INICIO_VAZIO = "2025-06-15"
DATA_FIM_VAZIO    = "2025-09-15"

RES = 10  # 10 m

# ===== Utilitários =====
def newest_item(items):
    if not items: return None
    # usa campo datetime do STAC
    return sorted(items, key=lambda it: it.datetime or datetime.fromisoformat(it.properties["datetime"]),
                  reverse=True)[0]

def affine_from_coords(x, y):
    res_x = float(x[1]-x[0]); res_y = float(y[1]-y[0])
    return Affine(res_x, 0, float(x.min()), 0, res_y, float(y.max()))

def save_geotiff(path, arr2d, x, y, compress=None):
    h, w = arr2d.shape
    transform = affine_from_coords(x, y)
    profile = {
        "driver":"GTiff","height":h,"width":w,"count":1,
        "dtype":"float32","crs":"EPSG:4326","transform":transform,
        "tiled": True, "compress": compress  # compress=None é mais rápido
    }
    with rasterio.open(path, "w", **profile) as dst:
        dst.write(arr2d.astype("float32"), 1)

# ===== 1) Buscar S2 mais recente no período =====
cat = Client.open(STAC_URL, headers=[], ignore_conformance=True)
items = list(cat.search(
    collections=["sentinel-2-l2a"],
    datetime=f"{DATA_INICIO_VAZIO}/{DATA_FIM_VAZIO}",
    intersects=AOI_POLY,
    query={"eo:cloud_cover":{"lt":80}},
    limit=1000
).get_items())

item = newest_item(items)
if item is None:
    raise SystemExit("Não achei cena S2 no período/área. Amplie datas/bbox ou suba eo:cloud_cover.")

# ===== 2) Empilhar só as bandas necessárias e recortar pelo bbox =====
# Earth Search assets: red(B04), nir(B08), scl(mask). Se 'scl' faltar, segue sem máscara.
assets_wanted = ["red","nir","scl"] if "scl" in item.assets else ["red","nir"]
da = stackstac.stack([item], assets=assets_wanted, bounds_latlon=AOI_BBOX,
                     epsg=4326, resolution=RES, chunksize=1024).transpose("time","y","x","band")

# remover a dimensão time (tem só 1 cena)
da = da.isel(time=0, drop=True)

# ===== 3) Máscara simples com SCL (se existir) e escala segura =====
bands = list(da.band.values)
mask = xr.ones_like(da.isel(band=0), dtype=bool)
if "scl" in bands:
    scl = da.sel(band="scl")
    # mantém vegetação/solo/água; remove sombra/nuvem/neve
    scl_ok = (scl > 3) & (~scl.isin([8,9,10,11]))
    mask = mask & scl_ok

red = da.sel(band="red"); nir = da.sel(band="nir")
# Se vier em 0..10000, divide por 10000. Fazemos teste rápido pelo máximo.
if float(red.max(skipna=True)) > 1.0:
    red = red / 10000.0
if float(nir.max(skipna=True)) > 1.0:
    nir = nir / 10000.0

red = red.where(mask); nir = nir.where(mask)

# NDVI
ndvi = (nir - red) / (nir + red)

# ===== 4) Salvar GeoTIFF =====
x = ndvi.x.values
y = ndvi.y.values
save_geotiff("ndvi_vazio.tif", ndvi.values, x, y, compress=None)  # mais rápido

print("OK: ndvi_vazio.tif salvo.")


/home/astolfinho/Documentos/vazio_sanitario/venv/lib/python3.12/site-packages/pystac_client/client.py:181: FutureWarning: The `ignore_conformance` option is deprecated and will be removed in the next major release. Instead use `set_conforms_to` or `add_conforms_to` to control behavior.
  warnings.warn(
/home/astolfinho/Documentos/vazio_sanitario/venv/lib/python3.12/site-packages/pystac_client/item_search.py:925: FutureWarning: get_items() is deprecated, use items() instead
  warnings.warn(


KeyboardInterrupt: 